In [20]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Reshape, merge, RepeatVector
from keras.layers.convolutional import Convolution2D, MaxPooling2D


batch_size = 10
nb_landmarks = 5
nb_epoch = 100
#nb_validation_data = (X_test, _Y_test)
nb_validation_data = None

nb_submodels = 3

# input image dimensions
img_chns, img_rows, img_cols = 3, 20, 20
# number of convolutional filters to use
nb_filters = [32, 64]
# size of pooling area for max pooling
nb_pool_sizes = [(2, 2), (2, 2)]
# convolution kernel size
nb_conv = 3
# activator
nb_activator = 'tanh'
# number of fully connected neurons in the penultimate layer
nb_penu_neurons = 128
# size of output vector, four "corner values" for each landmark
nb_output_size = nb_landmarks * 4

input_img = Input(shape=(img_chns, img_rows, img_cols), dtype='float32', name='input_img')

def gen_cnn(x) :
    x = (Convolution2D(nb_filters[0], nb_conv, nb_conv))(x)
    x = (Activation(nb_activator))(x)
    x = (MaxPooling2D(pool_size=nb_pool_sizes[0]))(x)
    x = (Convolution2D(nb_filters[1], nb_conv, nb_conv))(x)
    x = (Activation(nb_activator))(x)
    x = (MaxPooling2D(pool_size=nb_pool_sizes[1]))(x)
    x = (Dropout(0.25))(x)
    x = (Flatten())(x)
    return x

mask_cnn = gen_cnn(input_img)
x = (Dense(nb_penu_neurons))(mask_cnn)
x = (Activation(nb_activator))(x)
x = (Dropout(0.5))(x)
x = Dense(img_rows * img_cols)(x)
x = Activation('softmax')(x)

mask = (x)
masked_img = merge([input_img, 
                      Reshape((img_chns, img_rows, img_cols))(RepeatVector(3)((mask)))], 
                     mode='mul')

shared_cnn = Model(input=input_img, output=gen_cnn(input_img))
sub_output = shared_cnn(masked_img)
#sub_output = merge(sub_outputs, mode='concat')

x = (Dense(nb_penu_neurons))(sub_output)
x = (Activation(nb_activator))(x)
x = (Dropout(0.5))(x)
x = (Dense(nb_landmarks * 2))(x)
output_landmarks = Reshape((5,2))(x)

model = Model(input=input_img, output=output_landmarks)

model.compile(loss='mse',
              optimizer='adadelta',
              metrics=['accuracy'])

hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=nb_validation_data)

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])



Epoch 1/100
6402/6402 [==============================] - 12s - loss: 0.0146 - acc: 0.7555    
Epoch 2/100
6402/6402 [==============================] - 11s - loss: 0.0090 - acc: 0.7877    
Epoch 3/100
6402/6402 [==============================] - 11s - loss: 0.0084 - acc: 0.7951    
Epoch 4/100
6402/6402 [==============================] - 11s - loss: 0.0081 - acc: 0.7982    
Epoch 5/100
6402/6402 [==============================] - 11s - loss: 0.0080 - acc: 0.8029    
Epoch 6/100
6402/6402 [==============================] - 11s - loss: 0.0077 - acc: 0.8036    
Epoch 7/100
6402/6402 [==============================] - 11s - loss: 0.0071 - acc: 0.8181    
Epoch 8/100
6402/6402 [==============================] - 11s - loss: 0.0060 - acc: 0.8361    
Epoch 9/100
6402/6402 [==============================] - 11s - loss: 0.0047 - acc: 0.8541    
Epoch 10/100
6402/6402 [==============================] - 11s - loss: 0.0041 - acc: 0.8647    
Epoch 11/100
6402/6402 [==============================] - 1

In [8]:
# save model
def save_model(model_filename) :
    print(model.to_json(), file=open(model_filename+'.json', 'w'))
    model.save_weights(model_filename+'.weight')

In [9]:
save_model('../model/all(ratio-0.8,rand-1337),single-mask_single,20px,500ep')

In [10]:
Y_pred = model.predict(X_test)

In [13]:
# observe learning results
import random
random.seed(1337)
l = list(random.sample(range(len(Y_test)), 30))
plt.figure()
for iplt, i in enumerate(l) :
    plt.subplot(5, 6, iplt+1)
    img = X_test[i].transpose((1,2,0))
    io.imshow(img)
    pts2 = Y_test[i]
    plt.plot(pts2[:,0]*img_rows, pts2[:,1]*img_cols, 'o')
    pts = Y_pred[i]
    plt.plot(pts[:,0]*img_rows, pts[:,1]*img_cols, 'ro')
    plt.axis('off')
plt.tight_layout()
io.show()

In [1]:
from face_data import *
X, Y = data('../../../result_20/', subdirs)
(X_train, Y_train), (X_test, Y_test) = split_data(X, Y, ratio_train=0.8, rand_seed=1337)

X shape: (8003, 3, 20, 20)
Y shape: (8003, 5, 2)
X_train shape: (6402, 3, 20, 20)
Y_train shape: (6402, 5, 2)
6402 train samples
1601 test samples


In [10]:
# load model
model_filename = '../model/all(ratio-0.8,rand-1337),one-more-Dense,20px,500ep'
import keras.models
model2 = keras.models.model_from_json(open(model_filename+'.json').read())
model2.load_weights(model_filename+'.weight')
model2.compile(loss='mse',
              optimizer='adadelta',
              metrics=['accuracy'])

In [15]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.00114242155911
Test accuracy: 0.939662715333


In [14]:
model=model2